In [186]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [187]:
import os

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [188]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [189]:
 ##parameters definition
 
 input_size = 784 # 28x28
 hidden_size = 500 
 num_classes = 10
 num_epochs = 3
 batch_size = 100
 learning_rate = 0.01 

In [190]:
# Loading the MNIST dataset 
train_dataset = datasets.MNIST(root='./data', 
                                            train=True, 
                                       transform=transforms.ToTensor(),  
                                           download=True)
test_dataset = datasets.MNIST(root='./data', 
                                           train=False, 
                                           transform=transforms.ToTensor()) 

In [191]:
# Importing Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False) 

In [192]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        ## cnn layers 
        self.conv1 = nn.Sequential(         
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
       
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        
        x = x.reshape(x.shape[0], -1) 

        output = self.out(x)
        return output, x    # return x for visualization

In [193]:
#Defining our model 

model = LeNet().to(device)
model

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [194]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [195]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(num_epochs + 1)]

In [196]:
  def train(epoch):

    model.train()
    #n_total_steps = len(train_loader)
  
    
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs[0], labels)

        # Backward and optimize
        
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, i * len(images), len(train_loader.dataset),100. * i / len(train_loader), loss.item()))
          train_losses.append(loss.item())
          train_counter.append((i*64) + ((epoch-1)*len(train_loader.dataset)))
          torch.save(model.state_dict(), '/content/results/model.pth')
    


In [197]:
def test():

  model.eval()
  
  n_correct = 0
  test_loss = 0

  with torch.no_grad():
     
     for images, labels in test_loader:
         images = images.to(device)
         labels = labels.to(device)

         outputs = model(images)
         test_loss += F.nll_loss(outputs[0], labels, size_average=False).item()
         # max returns (value ,index)

         predicted = outputs[0].data.max(1, keepdim=True)[1]

         n_correct += predicted.eq(labels.data.view_as(predicted)).sum()

     test_loss /= len(test_loader.dataset)
     test_losses.append(test_loss)
     print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, n_correct, len(test_loader.dataset), 100. * n_correct / len(test_loader.dataset)))

In [143]:
for x, y in train_loader:
  # print(x.size())
  out = model(x.to(device))
  #print(out[1].shape)
  print (out[0].data.max(1, keepdim=True)[1].shape)
  break

torch.Size([100, 1])


In [198]:
!mkdir results

mkdir: cannot create directory ‘results’: File exists


In [199]:
for epoch in range(1, num_epochs+1):
  train(epoch)
  test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301420
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.099836
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.122429
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.040324
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.070022
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.088839


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: -9.6525, Accuracy: 9829/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.062003
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.037575
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.076708
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.138790
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.067886
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.054056

Test set: Avg. loss: -10.8578, Accuracy: 9861/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.073194
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.015196
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.074773
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.155476
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.019139
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.152860

Test set: Avg. loss: -10.7685, Accuracy: 9825/10000 (98%)



In [201]:
loaded_model = LeNet().to(device)
loaded_model.load_state_dict(torch.load('/content/results/model.pth'))
loaded_model.eval()

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [224]:
for name, param in loaded_model.named_parameters():
  
  Hessian_params = torch.autograd.functional.hessian(loss, param,)

NameError: ignored

In [1]:
def my_loss(outputs, labels):
    loss = criterion(outputs, labels)
    return loss

for i, (images, labels) in enumerate(train_loader):  
      
      images = images.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = loaded_model(images)

Hess_params = torch.autograd.functional.hessian(my_loss, (outputs, labels))

NameError: ignored

In [253]:
 def compute_hessian():

  for i, (images, labels) in enumerate(train_loader):  
      
      images = images.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = loaded_model(images)
     

      for name, param in loaded_model.named_parameters():
        
        p = param
        loss = criterion(outputs[0], labels)
        print(loss)
       

        # Backward and optimize
        optimizer.zero_grad()
      
        loss.backward(retain_graph=True)
        
        grad_params = torch.autograd.grad(loss, p, create_graph=True,allow_unused=True)  # p is the weight matrix for a particular layer 
        
        hess_params = torch.zeros_like(grad_params[0])
        

        for i in range(grad_params[0].size(0)):
            for j in range(grad_params[0].size(1)):
              
              hess_params[i, j] = torch.autograd.grad(grad_params[0][i][j], p, retain_graph=True)[0][i, j]
              
        optimizer.step()

        return hess_params


In [ ]:
h=compute_hessian()

In [ ]:
def train_OBD(epoch):

  model.train()
  n_total_steps = len(train_loader)
  
    
  for i, (images, labels) in enumerate(train_loader):  
      # origin shape: [100, 1, 28, 28]
      # resized: [100, 784]
      images = images.reshape(-1, 28*28).to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = model(images)

      for name, param in model.named_parameters():
        
        p = param
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
      
        loss.backward(retain_graph=True)
        grad_params = torch.autograd.grad(loss, p, create_graph=True)  # p is the weight matrix for a particular layer 
        hess_params = torch.zeros_like(grad_params[0])

        for i in range(grad_params[0].size(0)):
            for j in range(grad_params[0].size(1)):
                hess_params[i, j] = torch.autograd.grad(grad_params[0][i][j], p, retain_graph=True)[0][i, j]
        optimizer.step()

        if i % 100 == 0:
          print('Train_OBD Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, i * len(images), len(train_loader.dataset),
            100. * i / len(train_loader), loss.item()))
          train_losses.append(loss.item())
          train_counter.append((i*64) + ((epoch-1)*len(train_loader.dataset)))


  return grad_params, hess_params

In [ ]:
for epoch in range(num_epochs):
  train_OBD(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/3], Step[500/600], Loss: 0.0179


RuntimeError: ignored